In [4]:
!pip install pandas scikit-learn


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
# Sample dataset
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 4, 4, 5],
    'book_id': ['A', 'B', 'C', 'A', 'C', 'B', 'D', 'A', 'D', 'C'],
    'rating': [5, 4, 3, 4, 5, 5, 3, 4, 5, 2]
}
df = pd.DataFrame(data)


In [7]:
# Create user-item matrix
user_item_matrix = df.pivot_table(index='user_id', columns='book_id', values='rating').fillna(0)


In [8]:
# Compute cosine similarity
cosine_sim = cosine_similarity(user_item_matrix)
similarity_df = pd.DataFrame(cosine_sim, index=user_item_matrix.index, columns=user_item_matrix.index)


In [9]:
def get_recommendations(user_id, user_item_matrix, similarity_df, n_recommendations=3):
    similar_users = similarity_df[user_id].sort_values(ascending=False).index[1:]  # Exclude the user themselves
    recommendations = pd.Series(dtype='float64')
    
    for similar_user in similar_users:
        # Get the books rated by similar users that the target user hasn't rated
        unrated_books = user_item_matrix.loc[user_id][user_item_matrix.loc[similar_user] > 0].index
        for book in unrated_books:
            if book not in recommendations:
                recommendations[book] = user_item_matrix.loc[similar_user][book] * similarity_df[user_id][similar_user]
    
    # Return top N recommendations
    return recommendations.nlargest(n_recommendations).index.tolist()


In [10]:
user_id = 1
recommended_books = get_recommendations(user_id, user_item_matrix, similarity_df)
print(f"Recommended books for User {user_id}: {recommended_books}")


Recommended books for User 1: ['C', 'A', 'B']


In [11]:
user_id = 2
recommended_books = get_recommendations(user_id, user_item_matrix, similarity_df)
print(f"Recommended books for User {user_id}: {recommended_books}")

Recommended books for User 2: ['A', 'B', 'D']


In [12]:
user_id = 3
recommended_books = get_recommendations(user_id, user_item_matrix, similarity_df)
print(f"Recommended books for User {user_id}: {recommended_books}")

Recommended books for User 3: ['A', 'D', 'B']
